In [1]:
!wget https://data.cityofnewyork.us/api/views/hvrh-b6nb/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*

--2023-10-11 09:04:46--  https://data.cityofnewyork.us/api/views/hvrh-b6nb/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [          <=>       ]   2.08G  1.75MB/s    in 16m 57s 

2023-10-11 09:21:43 (2.09 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [2229139706]



In [1]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.columns

In [ ]:
import pandas as pd
from itertools import combinations

df = pd.read_csv('data.csv')

quasi_identifiers = ['Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count']

k = 3

def find_k_anonymous_groups(df, quasi_identifiers, k):
    groups = []
    for _, group in df.groupby(quasi_identifiers):
        if len(group) >= k:
            groups.append(group)
    return groups


k_anonymous_groups = find_k_anonymous_groups(df, quasi_identifiers, k)


k_anonymized_df = pd.concat(k_anonymous_groups)

k_anonymized_df.to_csv('k_anonymized_dataset_full.csv', index=False)


In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv('k_anonymized_dataset_full.csv')

numerical_attributes = ['Trip_distance', 'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Total_amount']
for attr in numerical_attributes:
    df[attr] = pd.to_numeric(df[attr], errors='coerce')  # Convert to float, coerce errors to NaN
    df[attr] = (df[attr] - df[attr].min()) / (df[attr].max() - df[attr].min())


quasi_identifiers = ['Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count']
sensitive_attribute = 'Fare_amount'

L = 2

def generalize_fare(fare):
    return round(fare)

def apply_l_diversity(group_data):
    unique_sensitive_values = group_data[sensitive_attribute].unique()
    if len(unique_sensitive_values) < L:
        print(f'{unique_sensitive_values} of {L}\n')
        group_data[sensitive_attribute] = generalize_fare(group_data[sensitive_attribute])
    return group_data


grouped = df.groupby(quasi_identifiers)

df_anonymized = grouped.apply(apply_l_diversity).reset_index(drop=True)

print('\nDone')
df_anonymized.to_csv('k_anonymized_l_diversity_dataset.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv('k_anonymized_dataset_full.csv')

# Define your quasi-identifiers and sensitive attribute
quasi_identifiers = ['Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count']
sensitive_attribute = 'Fare_amount'

# Define L-diversity parameter
L = 3  # Adjust according to your privacy requirements

# Function to apply L-diversity with data perturbation
def apply_l_diversity_with_perturbation(group_data):
    unique_sensitive_values = group_data[sensitive_attribute].unique()
    num_unique_sensitive_values = len(unique_sensitive_values)

    if num_unique_sensitive_values < L:
        # Convert the sensitive attribute to a numeric type
        group_data[sensitive_attribute] = pd.to_numeric(group_data[sensitive_attribute], errors='coerce')

        # Apply data perturbation to achieve L-diversity
        if num_unique_sensitive_values > 0:
            # Calculate the amount of noise to add
            noise_std_dev = (group_data[sensitive_attribute].max() - group_data[sensitive_attribute].min()) / 10  # Adjust as needed

            # Generate random noise
            noise = np.random.normal(scale=noise_std_dev, size=len(group_data))

            # Perturb the sensitive attribute with noise
            group_data[sensitive_attribute] = group_data[sensitive_attribute] + noise

        # Continue with other transformations or suppressions as needed

    return group_data

# Group the data by quasi-identifiers
grouped = df.groupby(quasi_identifiers)



# Apply L-diversity with data perturbation to each group
df_anonymized = grouped.apply(apply_l_diversity_with_perturbation).reset_index(drop=True)

# Evaluate and Validate
# Perform re-identification risk analysis and statistical tests

# Post-Processing (if needed)
# Further data cleaning or adjustments

# Save the anonymized dataset
df_anonymized.to_csv('l_diversity_with_perturbation_dataset.csv', index=False)


C:\Users\OMEN\AppData\Local\Temp\ipykernel_6784\3856959818.py:44: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_anonymized = grouped.apply(apply_l_diversity_with_perturbation).reset_index(drop=True)


In [3]:
import dask.dataframe as dd
import pandas as pd

# Sample dataset with quasi-identifiers (e.g., age, gender, and ZIP code)
data = {
    'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'Age': [25, 32, 25, 45, 27, 32, 45, 27, 25],
    'Gender': ['M', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'F'],
    'ZIP': ['12345', '54321', '12345', '67890', '12345', '54321', '67890', '12345', '54321']
}

df = pd.DataFrame(data)

# Define the value of k for k-anonymity
k = 3

# Function to perform generalization (e.g., by replacing ZIP codes with a general region)
def generalize_zip(zip_code):
    return zip_code[:3] + "XXX"

# Initialize a new column for generalized ZIP codes
df['GeneralizedZIP'] = df['ZIP'].apply(generalize_zip)

# Use Dask to read and process the data in chunks
ddf = dd.from_pandas(df, npartitions=2)  # Set the number of partitions for parallel processing

def k_anonymize(partition):
    # Group the partitioned data by quasi-identifiers and count the occurrences
    grouped = partition.groupby(['Age', 'Gender', 'GeneralizedZIP']).size().reset_index(name='Count')
    # Identify groups that do not meet the k-anonymity requirement
    non_k_anonymous_groups = grouped[grouped['Count'] < k]

    # Anonymize the non-k-anonymous groups in this partition
    for idx, row in non_k_anonymous_groups.iterrows():
        mask = (partition['Age'] == row['Age']) & (partition['Gender'] == row['Gender']) & (partition['GeneralizedZIP'] == row['GeneralizedZIP'])
        partition.loc[mask, 'GeneralizedZIP'] = "General"
    return partition

# Apply k-anonymization to each partition
anonymized_ddf = ddf.map_partitions(k_anonymize)

# Compute and store the result to a new Dask DataFrame
anonymized_df = anonymized_ddf.compute()

# Display the anonymized dataset
print(anonymized_df)


   Age Gender GeneralizedZIP  Count
0    1    foo            foo      2
   Age Gender GeneralizedZIP  Count
0   25      M         123XXX      2
1   27      F         123XXX      1
2   32      F         543XXX      1
3   45      M         678XXX      1   Age Gender GeneralizedZIP  Count
0   25      F         543XXX      1
1   27      M         123XXX      1
2   32      F         543XXX      1
3   45      M         678XXX      1

   ID  Age Gender    ZIP GeneralizedZIP
0   1   25      M  12345        General
1   2   32      F  54321        General
2   3   25      M  12345        General
3   4   45      M  67890        General
4   5   27      F  12345        General
5   6   32      F  54321        General
6   7   45      M  67890        General
7   8   27      M  12345        General
8   9   25      F  54321        General
